## Step 1: Verify Dual GPU Environment

Verifies dual T4 GPU setup for split-GPU operation where GPU 0 will run llama-server for LLM inference and GPU 1 will handle RAPIDS/Graphistry graph analytics.

In [1]:
import subprocess
import os

print("="*70)
print("🔍 SPLIT-GPU ENVIRONMENT CHECK")
print("="*70)

# Check GPUs
result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for split-GPU operation!")
    print("   GPU 0 → llama-server (LLM)")
    print("   GPU 1 → RAPIDS/Graphistry")
else:
    print("\n⚠️ Need 2 GPUs for split operation")

🔍 SPLIT-GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 15096 MiB
   1, Tesla T4, 15360 MiB, 15096 MiB

✅ Dual T4 ready for split-GPU operation!
   GPU 0 → llama-server (LLM)
   GPU 1 → RAPIDS/Graphistry


## Step 2: Install Dependencies

Installs llamatelemetry v0.1.0, cuGraph (RAPIDS 25.6.0 matching Kaggle), and Graphistry for the split-GPU workflow combining LLM inference with graph visualization.

In [2]:
%%time
print("📦 Installing dependencies...")

# Install llamatelemetry v0.1.0 (force fresh install to ensure correct binaries)
!pip install -q --no-cache-dir git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0

# Install cuGraph (matching Kaggle RAPIDS 25.6.0)
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*"

# Install Graphistry
!pip install -q graphistry

# Verify installations
import llamatelemetry
print(f"\n✅ llamatelemetry {llamatelemetry.__version__} installed")

try:
    import cudf
    import cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 40.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 46.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.8/439.8 kB 10.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Kaggle

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.(…):   0%|          | 0.00/1.01G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 0 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!


✅ llamatelemetry 0.1.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.4
CPU times: user 42.9 s, sys: 10.2 s, total: 53.1 s
Wall time: 1min 15s


## Add Graphistry and/or Huggginface Secrets in Kaggle

Configures Graphistry API credentials from Kaggle secrets to enable cloud-based graph visualization rendering and sharing capabilities.

In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
graphistry.register(
    api=3,
    protocol="https",
    server="hub.graphistry.com",
    personal_key_id=user_secrets.get_secret("Graphistry_Personal_Key_ID"),
    personal_key_secret=user_secrets.get_secret("Graphistry_Personal_Secret_Key")
)

## Step 3: Download GGUF Model

Downloads Llama 3.2 3B Q4_K_M GGUF model for deployment on GPU 0, checking for FlashAttention support in llamatelemetry binaries for optimal performance.

In [4]:
%%time
from huggingface_hub import hf_hub_download
import os

# Try a Llama 3 GGUF model - some builds include FlashAttention
MODEL_REPO = "bartowski/Llama-3.2-3B-Instruct-GGUF"
MODEL_FILE = "Llama-3.2-3B-Instruct-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")
print(f"   Note: Check if your llamatelemetry binary supports FlashAttention")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")
print(f"   Note: You'll need to verify FlashAttention support in llamatelemetry")

📥 Downloading Llama-3.2-3B-Instruct-Q4_K_M.gguf...
   Note: Check if your llamatelemetry binary supports FlashAttention


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf
   Size: 1.88 GB
   Note: You'll need to verify FlashAttention support in llamatelemetry
CPU times: user 3.83 s, sys: 7.53 s, total: 11.4 s
Wall time: 4.51 s


## Step 4: Start llama-server on GPU 0 Only

Starts llama-server exclusively on GPU 0 using tensor-split configuration (1.0,0.0), reserving GPU 1 completely free for RAPIDS/Graphistry workloads.

In [5]:
from llamatelemetry.server import ServerManager

print("="*70)
print("🚀 STARTING LLAMA-SERVER ON GPU 0")
print("="*70)

# Configuration for GPU 0 ONLY (leave GPU 1 for RAPIDS)
print("\n📋 Configuration:")
print("   GPU 0: 100% (llama-server)")
print("   GPU 1: 0% (reserved for RAPIDS)")

server = ServerManager()
server.start_server(
    model_path=model_path,
    host="127.0.0.1",
    port=8090,
    
    # GPU 0 only configuration
    gpu_layers=99,
    tensor_split="1.0,0.0",  # 100% on GPU 0, 0% on GPU 1
    
    # Optimize for single GPU
    ctx_size=4096,
    # Remove or set to False: flash_attention=False,
)

if server.check_server_health():
    print("\n✅ llama-server running on GPU 0!")
else:
    print("\n❌ Server failed to start")

🚀 STARTING LLAMA-SERVER ON GPU 0

📋 Configuration:
   GPU 0: 100% (llama-server)
   GPU 1: 0% (reserved for RAPIDS)
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready........ ✓ Ready in 5.0s

✅ llama-server running on GPU 0!


## Step 5: Verify GPU Split

Verifies the GPU memory split by checking VRAM usage, confirming GPU 0 is running the LLM while GPU 1 remains free with >14GB available for graph analytics.

In [6]:
print("="*70)
print("📊 GPU MEMORY SPLIT VERIFICATION")
print("="*70)

!nvidia-smi --query-gpu=index,name,memory.used,memory.free --format=csv

import subprocess
result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"],
    capture_output=True, text=True
)

lines = result.stdout.strip().split('\n')
if len(lines) >= 2:
    gpu1_free = int(lines[1].split(',')[1].strip())
    print(f"\n✅ GPU 1 has {gpu1_free} MiB free for RAPIDS!")

📊 GPU MEMORY SPLIT VERIFICATION
index, name, memory.used [MiB], memory.free [MiB]
0, Tesla T4, 2563 MiB, 12533 MiB
1, Tesla T4, 103 MiB, 14993 MiB

✅ GPU 1 has 14993 MiB free for RAPIDS!


## Step 6: Initialize RAPIDS on GPU 1

Forces RAPIDS (cuDF/cuGraph) to use GPU 1 by setting CUDA_VISIBLE_DEVICES environment variable and creates a test DataFrame to verify GPU assignment.

In [7]:
import os
# Force RAPIDS to use GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("="*70)
print("🔥 INITIALIZING RAPIDS ON GPU 1")
print("="*70)

import cudf
import cupy as cp

# Verify we're on the right GPU
print(f"\n📊 RAPIDS GPU Info:")
device = cp.cuda.Device(0)  # Device 0 in filtered view = actual GPU 1
print(f"   Device: {device.id} (filtered view)")
print(f"   Actual GPU: 1 (Tesla T4)")

# Test cuDF on GPU 1
test_df = cudf.DataFrame({
    'source': [0, 1, 2, 3, 4],
    'target': [1, 2, 3, 4, 0],
    'weight': [1.0, 2.0, 1.5, 0.5, 3.0]
})

print(f"\n✅ cuDF working on GPU 1")
print(f"   Test DataFrame: {test_df.shape}")

🔥 INITIALIZING RAPIDS ON GPU 1

📊 RAPIDS GPU Info:
   Device: 0 (filtered view)
   Actual GPU: 1 (Tesla T4)

✅ cuDF working on GPU 1
   Test DataFrame: (5, 3)


## Step 7: Create Sample Graph Data

Creates a sample social network graph with 10 nodes and 16 edges using cuDF DataFrames on GPU 1, building a cuGraph object for GPU-accelerated analytics.

In [8]:
import cudf
import cugraph

print("="*70)
print("📊 CREATING SAMPLE GRAPH ON GPU 1")
print("="*70)

# Create a sample social network graph
edges = cudf.DataFrame({
    'source': [0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9],
    'target': [1, 2, 3, 2, 4, 3, 5, 4, 6, 5, 6, 7, 7, 8, 9, 0],
})

# Node labels
node_names = {
    0: "Alice", 1: "Bob", 2: "Charlie", 3: "Diana",
    4: "Eve", 5: "Frank", 6: "Grace", 7: "Henry",
    8: "Ivy", 9: "Jack"
}

print(f"\n📊 Graph created:")
print(f"   Nodes: {len(node_names)}")
print(f"   Edges: {len(edges)}")

# Create cuGraph graph
G = cugraph.Graph()
G.from_cudf_edgelist(edges, source='source', destination='target')

print(f"\n✅ cuGraph graph created on GPU 1")

📊 CREATING SAMPLE GRAPH ON GPU 1

📊 Graph created:
   Nodes: 10
   Edges: 16

✅ cuGraph graph created on GPU 1


## Step 8: Run Graph Analytics on GPU 1

Runs GPU-accelerated graph analytics on GPU 1 computing PageRank and Betweenness Centrality metrics to identify influential nodes and network connectors.

In [9]:
print("="*70)
print("🔬 GPU-ACCELERATED GRAPH ANALYTICS")
print("="*70)

# PageRank
print("\n📊 PageRank Analysis:")
pagerank = cugraph.pagerank(G)
pagerank = pagerank.sort_values('pagerank', ascending=False)

for _, row in pagerank.to_pandas().head(5).iterrows():
    node_id = int(row['vertex'])
    score = row['pagerank']
    name = node_names.get(node_id, f"Node {node_id}")
    print(f"   {name}: {score:.4f}")

# Betweenness Centrality
print("\n📊 Betweenness Centrality:")
bc = cugraph.betweenness_centrality(G)
bc = bc.sort_values('betweenness_centrality', ascending=False)

for _, row in bc.to_pandas().head(5).iterrows():
    node_id = int(row['vertex'])
    score = row['betweenness_centrality']
    name = node_names.get(node_id, f"Node {node_id}")
    print(f"   {name}: {score:.4f}")

print("\n✅ Graph analytics computed on GPU 1")

🔬 GPU-ACCELERATED GRAPH ANALYTICS

📊 PageRank Analysis:
   Alice: 0.1219
   Frank: 0.1204
   Diana: 0.1185
   Charlie: 0.1177
   Henry: 0.0984

📊 Betweenness Centrality:


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


   Alice: 0.2093
   Frank: 0.1824
   Henry: 0.1389
   Diana: 0.1324
   Charlie: 0.1083

✅ Graph analytics computed on GPU 1


## Step 9: Use LLM to Analyze Graph Results

Demonstrates simultaneous GPU operation by using the LLM on GPU 0 to analyze PageRank results from graph analytics computed on GPU 1, combining AI reasoning with graph insights.

In [10]:
# Step 9: Use LLM to Analyze Graph Results with llamatelemetry
from llamatelemetry.api.client import LlamaCppClient

print("="*70)
print("🤔 LLM ANALYSIS OF GRAPH RESULTS")
print("="*70)

# Get top PageRank nodes
top_nodes = pagerank.to_pandas().head(3)
top_names = [node_names[int(row['vertex'])] for _, row in top_nodes.iterrows()]

# Create prompt for LLM
prompt = f"""I have a social network graph with 10 people. 
The PageRank analysis shows the most influential people are: {', '.join(top_names)}.

Based on this, what insights can you provide about the network structure? 
Keep your response to 3-4 sentences."""

# Connect to llama-server running on port 8090
client = LlamaCppClient(base_url="http://127.0.0.1:8090")

response = client.chat.create(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=200,
    temperature=0.7
)

print(f"\n📋 LLM Analysis (GPU 0):")
print(response.choices[0].message.content)

print("\n✅ Simultaneous GPU operation:")
print("   GPU 0: LLM inference")
print("   GPU 1: Graph analytics (previously computed)")

🤔 LLM ANALYSIS OF GRAPH RESULTS

📋 LLM Analysis (GPU 0):
{"name": "analyze_network", "parameters": {"graph": "[[1, 2, 0], [2, 1, 3], [0, 3, 1]]"}}

✅ Simultaneous GPU operation:
   GPU 0: LLM inference
   GPU 1: Graph analytics (previously computed)


## Step 10: Graphistry Visualization Setup

Prepares graph data for Graphistry visualization by converting cuDF to pandas, merging PageRank and betweenness metrics with node information, and displaying computed metrics.

In [11]:
import graphistry
import pandas as pd

print("="*70)
print("📊 GRAPHISTRY VISUALIZATION")
print("="*70)

# Convert to pandas for Graphistry (works in-notebook)
edges_pd = edges.to_pandas()
edges_pd['source_name'] = edges_pd['source'].map(node_names)
edges_pd['target_name'] = edges_pd['target'].map(node_names)

# Create nodes DataFrame with metrics
pagerank_pd = pagerank.to_pandas()
bc_pd = bc.to_pandas()

nodes_pd = pd.DataFrame({
    'node_id': list(node_names.keys()),
    'name': list(node_names.values())
})
nodes_pd = nodes_pd.merge(
    pagerank_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id'
)
nodes_pd = nodes_pd.merge(
    bc_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id'
)

print(f"\n📊 Prepared for visualization:")
print(f"   Nodes: {len(nodes_pd)}")
print(f"   Edges: {len(edges_pd)}")

# Note: Graphistry requires registration for full visualization
# For demo purposes, we'll show the prepared data
print(f"\n📋 Node Metrics:")
print(nodes_pd[['name', 'pagerank', 'betweenness_centrality']].to_string(index=False))

📊 GRAPHISTRY VISUALIZATION

📊 Prepared for visualization:
   Nodes: 10
   Edges: 16

📋 Node Metrics:
   name  pagerank  betweenness_centrality
  Alice  0.121906                0.209259
    Bob  0.091935                0.037037
Charlie  0.117719                0.108333
  Diana  0.118467                0.132407
    Eve  0.091814                0.057407
  Frank  0.120436                0.182407
  Grace  0.093641                0.060185
  Henry  0.098380                0.138889
    Ivy  0.073543                0.064815
   Jack  0.072161                0.092593


## Step 11: Interactive LLM + Graph Workflow

Creates an interactive workflow combining LLM and graph analytics by using the LLM to interpret individual node metrics (PageRank, betweenness) and explain their network roles.

In [12]:
print("="*70)
print("🔄 INTERACTIVE LLM + GRAPH WORKFLOW")
print("="*70)

def analyze_node(node_name):
    """Use LLM to analyze a specific node's network position."""
    node_data = nodes_pd[nodes_pd['name'] == node_name].iloc[0]
    
    prompt = f"""Analyze the network position of {node_name}:
    - PageRank score: {node_data['pagerank']:.4f} (higher = more influential)
    - Betweenness centrality: {node_data['betweenness_centrality']:.4f} (higher = more connections)
    
    What does this tell us about {node_name}'s role in the network? Answer in 2 sentences."""
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Analyze top 3 nodes
print("\n🔍 Node Analysis:")
for name in ['Alice', 'Charlie', 'Frank']:
    print(f"\n📌 {name}:")
    analysis = analyze_node(name)
    print(f"   {analysis}")

🔄 INTERACTIVE LLM + GRAPH WORKFLOW

🔍 Node Analysis:

📌 Alice:
   {"name": "analyze_network_position", "parameters": {"pr_score": "0.1219", "betweenness_centrality": "0.2093"}}

📌 Charlie:
   {"name": "analyze_network_position", "parameters": {"pr_score": "0.1177", "betweenness_centrality": "0.1083"}}

📌 Frank:
   {"name": "analyze_network_position", "parameters": {"pr_score": "0.1204", "betweenness_centrality": "0.1824"}}


## Step 12: Monitor Both GPUs

Monitors both GPUs using nvidia-smi to display memory usage and confirm the split-GPU architecture is working with LLM on GPU 0 and RAPIDS on GPU 1.

In [13]:
print("="*70)
print("📊 DUAL GPU MONITORING")
print("="*70)

!nvidia-smi

print("\n💡 Split-GPU Operation:")
print("   GPU 0: llama-server (GGUF model loaded)")
print("   GPU 1: RAPIDS memory (cuDF/cuGraph data structures)")

📊 DUAL GPU MONITORING
Tue Jan 20 21:07:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P0             34W /   70W |    2677MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------

## Step 13: Graphistry Dashboard

Creates an advanced Graphistry dashboard with community detection (Louvain), node role classification (Hub/Bridge/Influencer), and rich visual encodings (color, size, icons) for interactive exploration.

In [14]:
print("="*70)
print("🎨 SOPHISTICATED GRAPHISTRY DASHBOARD")
print("="*70)

# Advanced Graphistry visualization showcasing llamatelemetry v0.1.0 + RAPIDS integration
import graphistry
import webbrowser
import pandas as pd
import numpy as np

# Step 1: Compute additional graph metrics for richer visualization
print("\n📊 Computing additional graph metrics...")

# Ensure edges_pd exists and is in pandas format
if 'edges_pd' not in locals() and 'edges_pd' not in globals():
    if 'edges' in locals() or 'edges' in globals():
        if hasattr(edges, 'to_pandas'):
            edges_pd = edges.to_pandas()
        else:
            edges_pd = edges.copy()
    else:
        raise ValueError("No edges data found. Please ensure 'edges' or 'edges_pd' exists.")

print(f"   Edge data shape: {edges_pd.shape}")

# Ensure nodes_pd exists and is in pandas format
if 'nodes_pd' not in locals() and 'nodes_pd' not in globals():
    if 'nodes' in locals() or 'nodes' in globals():
        if hasattr(nodes, 'to_pandas'):
            nodes_pd = nodes.to_pandas()
        else:
            nodes_pd = nodes.copy()
    else:
        raise ValueError("No nodes data found. Please ensure 'nodes' or 'nodes_pd' exists.")

print(f"   Node data shape: {nodes_pd.shape}")

# Add degree centrality
print("   Computing degree centrality...")
degree_in = edges_pd.groupby('target').size().reset_index(name='degree_in')
degree_out = edges_pd.groupby('source').size().reset_index(name='degree_out')

# Ensure 'node_id' column exists in nodes_pd
if 'node_id' not in nodes_pd.columns:
    # Try to find alternative ID column
    possible_id_cols = ['id', 'Id', 'ID', 'vertex', 'vertex_id', 'name']
    for col in possible_id_cols:
        if col in nodes_pd.columns:
            nodes_pd = nodes_pd.rename(columns={col: 'node_id'})
            print(f"   Renamed column '{col}' to 'node_id'")
            break
    else:
        # If no ID column found, create one
        nodes_pd['node_id'] = range(len(nodes_pd))
        print("   Created 'node_id' column")

# Rename degree columns for merging
degree_in = degree_in.rename(columns={'target': 'node_id'})
degree_out = degree_out.rename(columns={'source': 'node_id'})

# Merge degree columns
nodes_pd = nodes_pd.merge(degree_in, on='node_id', how='left')
nodes_pd = nodes_pd.merge(degree_out, on='node_id', how='left')

# Fill NaN values with 0
nodes_pd['degree_in'] = nodes_pd['degree_in'].fillna(0).astype(int)
nodes_pd['degree_out'] = nodes_pd['degree_out'].fillna(0).astype(int)
nodes_pd['total_degree'] = nodes_pd['degree_in'] + nodes_pd['degree_out']

print(f"   Degree stats - In: {nodes_pd['degree_in'].sum()}, Out: {nodes_pd['degree_out'].sum()}")

# Compute Louvain communities (if cuGraph is available)
try:
    print("\n   Running Louvain community detection...")
    import cugraph

    # Create cuGraph Graph from edges_pd
    if 'source' in edges_pd.columns and 'target' in edges_pd.columns:
        # Convert to cuDF if available
        try:
            import cudf
            edges_cudf = cudf.DataFrame({'source': edges_pd['source'], 
                                        'destination': edges_pd['target']})
            G_cugraph = cugraph.Graph()
            G_cugraph.from_cudf_edgelist(edges_cudf, source='source', destination='destination')
            
            louvain_result = cugraph.louvain(G_cugraph)
            
            if isinstance(louvain_result, tuple):
                communities, modularity = louvain_result
                print(f"   Modularity score: {modularity:.4f}")
            else:
                communities = louvain_result
                modularity = None
            
            # Convert to pandas
            communities_pd = communities.to_pandas()
            
            # Rename columns appropriately
            if 'vertex' in communities_pd.columns:
                communities_pd = communities_pd.rename(columns={'vertex': 'node_id', 'partition': 'community'})
            elif 'vertex_id' in communities_pd.columns:
                communities_pd = communities_pd.rename(columns={'vertex_id': 'node_id', 'partition_id': 'community'})
            
            # Merge with nodes_pd
            nodes_pd = nodes_pd.merge(communities_pd[['node_id', 'community']], 
                                     on='node_id', how='left')
            
            print(f"   Communities detected: {nodes_pd['community'].nunique()}")
            
        except Exception as e:
            print(f"   ⚠️ cuGraph processing failed: {e}")
            raise
    else:
        print("   ⚠️ Required 'source' or 'target' columns not found in edges_pd")
        raise ValueError("Missing required columns")
        
except Exception as e:
    print(f"   ⚠️ Louvain algorithm not available: {e}")
    print("   Using degree-based communities as fallback...")
    
    # Create simple communities based on degree
    nodes_pd['community'] = pd.qcut(nodes_pd['total_degree'], q=5, labels=False)
    modularity = None
    print(f"   Created {nodes_pd['community'].nunique()} communities based on degree")

# Add role classification based on metrics
print("\n   Computing node roles...")

# Ensure required metrics exist
for col in ['pagerank', 'betweenness_centrality']:
    if col not in nodes_pd.columns:
        print(f"   ⚠️ {col} not found, creating random values for demonstration")
        nodes_pd[col] = np.random.rand(len(nodes_pd))

def classify_role(row):
    pagerank = float(row['pagerank'])
    betweenness = float(row['betweenness_centrality'])
    
    if pagerank > 0.115 and betweenness > 0.15:
        return 'Hub'
    elif betweenness > 0.15:
        return 'Bridge'
    elif pagerank > 0.115:
        return 'Influencer'
    else:
        return 'Member'

nodes_pd['role'] = nodes_pd.apply(classify_role, axis=1)

print(f"   Role distribution:")
print(nodes_pd['role'].value_counts().to_string())

# Step 2: Prepare for Graphistry visualization
print("\n🎨 Preparing Graphistry visualization...")

# Ensure edges_pd has correct column names
edges_pd = edges_pd.rename(columns={'source': 'source', 'target': 'target'})

# Add node size based on PageRank
pr_min = nodes_pd['pagerank'].min()
pr_max = nodes_pd['pagerank'].max()
if pr_max > pr_min:
    nodes_pd['node_size'] = 20 + (nodes_pd['pagerank'] - pr_min) / (pr_max - pr_min) * 60
else:
    nodes_pd['node_size'] = 40

# Step 3: Create Graphistry visualization
print("   Creating Graphistry plot...")

# Bind the graph data
g = graphistry.bind(source="source", destination="target", node="node_id")
plotter = g.edges(edges_pd).nodes(nodes_pd)

# Apply visual encodings
print("   Applying visual encodings...")

# Community colors
community_colors = {
    0: '#FF6B6B',  # Red
    1: '#4ECDC4',  # Teal
    2: '#45B7D1',  # Blue
    3: '#FFA07A',  # Light Salmon
    4: '#98D8C8',  # Mint
}

# Map other communities to gray
for comm in nodes_pd['community'].unique():
    if comm not in community_colors and comm >= 0:
        community_colors[comm] = '#CCCCCC'

plotter = plotter.encode_point_color(
    'community',
    categorical_mapping=community_colors,
    default_mapping='#CCCCCC'
 )

# Apply size encoding
plotter = plotter.encode_point_size(
    'node_size'
 )

# Apply icon encoding
plotter = plotter.encode_point_icon(
    'role',
    categorical_mapping={
        'Hub': 'star',
        'Bridge': 'exchange-alt',
        'Influencer': 'trophy',
        'Member': 'user'
    },
    default_mapping='circle'
 )

# Configure visualization settings
print("   Configuring visualization settings...")
plotter = plotter.settings(url_params={
    'play': 0,
    'pointSize': 2.0,
    'edgeOpacity': 0.3,
    'showArrows': 'true',
    'showLabels': 'true',
    'menu': 'true',
    'info': 'true',
    'bg': '%23f5f5f5',  # Light background
})

# Step 4: Generate and open the visualization
print("\n🚀 Generating Graphistry dashboard...")

# Get the visualization URL
url = plotter.plot(render=False)

print("\n" + "="*70)
print("✅ SOPHISTICATED GRAPHISTRY DASHBOARD CREATED")
print("="*70)

print(f"\n🔗 Interactive Dashboard URL:")
print(f"   {url}")

# Try to open in a new browser tab (works in Jupyter/Kaggle environments)
print("\n🌐 Attempting to open in new browser tab...")
try:
    # For Kaggle/Jupyter notebooks
    from IPython.display import HTML, display
    display(HTML(f'<a href="{url}" target="_blank">Click here to open Graphistry dashboard in new tab</a>'))
    
    # Also try to open with webbrowser
    import webbrowser
    webbrowser.open_new_tab(url)
    print("   ✅ Dashboard should open in a new browser tab!")
except Exception as e:
    print(f"   ⚠️ Could not auto-open browser: {e}")
    print(f"   Please manually copy and paste the URL above into your browser")

# Display dashboard statistics
print("\n📊 Dashboard Features:")
print(f"   ✅ Node Count: {len(nodes_pd)}")
print(f"   ✅ Edge Count: {len(edges_pd)}")
print(f"   ✅ Communities: {nodes_pd['community'].nunique()}")
print(f"   ✅ Roles: {nodes_pd['role'].nunique()} distinct roles")

print(f"\n🎯 Visual Encodings:")
print(f"   • Color: Community membership")
print(f"   • Size: PageRank influence")
print(f"   • Icon: Network role (Hub/Bridge/Influencer/Member)")

print(f"\n🖱️ Interactive Features:")
print(f"   • Click and drag nodes")
print(f"   • Zoom with mouse wheel")
print(f"   • Hover for details")
print(f"   • Filter using control panel")

print("\n" + "="*70)
print("📋 Sample Data Preview:")
print("="*70)
print("\nNodes (first 5):")
print(nodes_pd[['node_id', 'community', 'role', 'total_degree']].head().to_string(index=False))
print("\nEdges (first 5):")
print(edges_pd[['source', 'target']].head().to_string(index=False))

print("\n" + "="*70)
print("🌐 OPEN THE URL ABOVE IN A NEW BROWSER TAB TO VIEW VISUALIZATION")
print("="*70)

🎨 SOPHISTICATED GRAPHISTRY DASHBOARD

📊 Computing additional graph metrics...
   Edge data shape: (16, 4)
   Node data shape: (10, 4)
   Computing degree centrality...
   Degree stats - In: 16, Out: 16

   Running Louvain community detection...
   Modularity score: 0.2578
   Communities detected: 3

   Computing node roles...
   Role distribution:
role
Member        6
Hub           2
Influencer    2

🎨 Preparing Graphistry visualization...
   Creating Graphistry plot...
   Applying visual encodings...
   Configuring visualization settings...

🚀 Generating Graphistry dashboard...

✅ SOPHISTICATED GRAPHISTRY DASHBOARD CREATED

🔗 Interactive Dashboard URL:
   https://hub.graphistry.com/graph/graph.html?dataset=a09aecdfcd7c4949be3320ff13e8e9a1&type=arrow&viztoken=f62a3329-7ea6-4009-a991-f3f938ccae3f&usertag=9150e829-pygraphistry-0.50.4&splashAfter=1768943283&info=true&play=0&pointSize=2.0&edgeOpacity=0.3&showArrows=true&showLabels=true&menu=true&bg=%23f5f5f5

🌐 Attempting to open in new br

   ✅ Dashboard should open in a new browser tab!

📊 Dashboard Features:
   ✅ Node Count: 10
   ✅ Edge Count: 16
   ✅ Communities: 3
   ✅ Roles: 3 distinct roles

🎯 Visual Encodings:
   • Color: Community membership
   • Size: PageRank influence
   • Icon: Network role (Hub/Bridge/Influencer/Member)

🖱️ Interactive Features:
   • Click and drag nodes
   • Zoom with mouse wheel
   • Hover for details
   • Filter using control panel

📋 Sample Data Preview:

Nodes (first 5):
 node_id  community       role  total_degree
       0          1        Hub             4
       1          1     Member             3
       2          1 Influencer             4
       3          1 Influencer             4
       4          1     Member             3

Edges (first 5):
 source  target
      0       1
      0       2
      0       3
      1       2
      1       4

🌐 OPEN THE URL ABOVE IN A NEW BROWSER TAB TO VIEW VISUALIZATION


Empty placeholder cell - can be removed or repurposed for additional code.

## Step 14: Cleanup

Stops llama-server, clears RAPIDS GPU memory, and displays final GPU status to confirm all resources have been released on both GPUs.

In [15]:
print("🛑 Stopping llama-server...")
server.stop_server()

# Clear RAPIDS memory
import gc
del G, edges, pagerank, bc
gc.collect()

print("\n✅ Resources cleaned up")
print("\n📊 Final GPU Status:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

🛑 Stopping llama-server...

✅ Resources cleaned up

📊 Final GPU Status:
index, memory.used [MiB], memory.free [MiB]
0, 109 MiB, 14987 MiB
1, 3 MiB, 15093 MiB


## 📚 Summary

### Split-GPU Architecture:
- **GPU 0**: llama-server with `tensor_split=[1.0, 0.0]`
- **GPU 1**: RAPIDS/cuGraph via `CUDA_VISIBLE_DEVICES="1"`

### Key Integration Points:
1. ✅ LLM for natural language analysis
2. ✅ cuGraph for GPU-accelerated graph algorithms
3. ✅ Graphistry for visualization
4. ✅ Combined insights from both

### Code Pattern:
```python
# GPU 0: llama-server
config = ServerConfig(tensor_split=[1.0, 0.0], main_gpu=0)

# GPU 1: RAPIDS
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import cudf, cugraph  # Uses GPU 1
```

---

**Next:** [07-openai-api-client](07-openai-api-client-llamatelemetry-v0.1.0.ipynb)

Empty placeholder cell - can be removed or repurposed for additional code.